In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-19 18:16:14.186648: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 18:16:14.219556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [4]:
from adapters import AutoAdapterModel
from tqdm import tqdm

from transformers import AutoConfig

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel")


In [6]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack

adapter_name= "stacked-adapter-task-modefied"
config = AdapterConfig.load("seq_bn", reduction_factor=16, leave_out=[])
model.add_adapter(adapter_name, config=config)
model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.train_adapter([adapter_name])
model.active_adapters = Stack(domain_adapter_coral, adapter_name)

fn.print_trainable_parameters(model)

trainable params: 1662813 || all params: 68472957 || trainable%: 2.428422946594814


In [7]:
model.active_adapters

Stack[domain_adapter_telephone-travel, stacked-adapter-task-modefied]

In [8]:
model.adapter_summary(as_dict=True)

[{'name': 'domain_adapter_telephone-travel',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': False,
  '%param': 0.6739671334336303},
 {'name': 'stacked-adapter-task-modefied',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': True,
  '%param': 0.6739671334336303},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [8]:
fn.print_trainable_parameters(model)
model.active_adapters

trainable params: 1662813 || all params: 68472957 || trainable%: 2.428422946594814


Stack[stacked-adapter-task-modefied]

In [9]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.33299595141700405
F1 score before adaptation: 0.2564812007826246


In [10]:

reload(fn)
trainer = fn.train_model(model,"stacked-modeified",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/23450 [00:00<?, ?it/s]

{'loss': 1.0753, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/121 [00:00<?, ?it/s]

{'eval_loss': 0.9973421096801758, 'eval_accuracy': 0.48377504848093084, 'eval_f1': 0.4398500410873099, 'eval_precision': 0.6497267007514298, 'eval_recall': 0.48377504848093084, 'eval_runtime': 10.4663, 'eval_samples_per_second': 739.042, 'eval_steps_per_second': 11.561, 'epoch': 0.21}
{'loss': 0.8813, 'learning_rate': 9.782135076252724e-05, 'epoch': 0.43}


...

{'eval_loss': 0.6725927591323853, 'eval_accuracy': 0.7211376858435682, 'eval_f1': 0.7218092631732027, 'eval_precision': 0.737583382635062, 'eval_recall': 0.7211376858435682, 'eval_runtime': 10.7085, 'eval_samples_per_second': 722.32, 'eval_steps_per_second': 11.299, 'epoch': 9.59}
{'loss': 0.5387, 'learning_rate': 1.96078431372549e-06, 'epoch': 9.81}


  0%|          | 0/121 [00:00<?, ?it/s]

{'eval_loss': 0.669899046421051, 'eval_accuracy': 0.7226890756302521, 'eval_f1': 0.7233711590541725, 'eval_precision': 0.7375439438077657, 'eval_recall': 0.7226890756302521, 'eval_runtime': 10.7099, 'eval_samples_per_second': 722.229, 'eval_steps_per_second': 11.298, 'epoch': 9.81}


Overwriting existing adapter 'domain_adapter_telephone-travel'.
Overwriting existing adapter 'stacked-adapter-task-modefied'.
Overwriting existing head 'stacked-adapter-task-modefied'


{'train_runtime': 2572.3451, 'train_samples_per_second': 291.613, 'train_steps_per_second': 9.116, 'train_loss': 0.6332235073585755, 'epoch': 10.0}


In [15]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7105263157894737
F1 score before adaptation: 0.7116428701675159


In [11]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7105263157894737
F1 score before adaptation: 0.7116428701675159


In [23]:
trainer.model.adapter_summary(as_dict=True)

[{'name': 'domain_adapter_telephone-travel',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': False,
  '%param': 0.6739671334336303},
 {'name': 'stacked-adapter-task-modefied',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': True,
  '%param': 0.6739671334336303},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [22]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name,with_head=True)

In [33]:
model.add_classification_head(
    "task-test-after-coral",
    num_labels=3,
  )

In [37]:
model.adapter_summary(as_dict=True)

[{'name': 'domain-adapter-coral',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': True,
  '%param': 0.6739671334336303},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [36]:
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.34210526315789475
F1 score before adaptation: 0.2640755557195107
